In [128]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
from skimage.transform import resize
from random import shuffle
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
# print(check_output(["ls", "train/*/*.jpg"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import matplotlib.pyplot as plt

In [102]:
for root, dirs, files in os.walk("./train", topdown=False):
    for name in files:
        print(os.path.join(root, name))

./train/iPhone-6/(iP6)269.jpg
./train/iPhone-6/(iP6)52.jpg
./train/iPhone-6/(iP6)237.jpg
./train/iPhone-6/(iP6)81.jpg
./train/iPhone-6/(iP6)202.jpg
./train/iPhone-6/(iP6)69.jpg
./train/iPhone-6/(iP6)134.jpg
./train/iPhone-6/(iP6)198.jpg
./train/iPhone-6/(iP6)143.jpg
./train/iPhone-6/(iP6)215.jpg
./train/iPhone-6/(iP6)47.jpg
./train/iPhone-6/(iP6)148.jpg
./train/iPhone-6/(iP6)189.jpg
./train/iPhone-6/(iP6)138.jpg
./train/iPhone-6/(iP6)32.jpg
./train/iPhone-6/(iP6)25.jpg
./train/iPhone-6/(iP6)29.jpg
./train/iPhone-6/(iP6)38.jpg
./train/iPhone-6/(iP6)267.jpg
./train/iPhone-6/(iP6)185.jpg
./train/iPhone-6/(iP6)256.jpg
./train/iPhone-6/(iP6)149.jpg
./train/iPhone-6/(iP6)60.jpg
./train/iPhone-6/(iP6)266.jpg
./train/iPhone-6/(iP6)93.jpg
./train/iPhone-6/(iP6)151.jpg
./train/iPhone-6/(iP6)128.jpg
./train/iPhone-6/(iP6)196.jpg
./train/iPhone-6/(iP6)159.jpg
./train/iPhone-6/(iP6)209.jpg
./train/iPhone-6/(iP6)18.jpg
./train/iPhone-6/(iP6)230.jpg
./train/iPhone-6/(iP6)37.jpg
./train/iPhone-6/(iP6)

./train/HTC-1-M7/(HTC-1-M7)219.jpg
./train/HTC-1-M7/(HTC-1-M7)181.jpg
./train/HTC-1-M7/(HTC-1-M7)28.jpg
./train/HTC-1-M7/(HTC-1-M7)228.jpg
./train/HTC-1-M7/(HTC-1-M7)139.jpg
./train/HTC-1-M7/(HTC-1-M7)218.jpg
./train/HTC-1-M7/(HTC-1-M7)258.jpg
./train/HTC-1-M7/(HTC-1-M7)69.jpg
./train/HTC-1-M7/(HTC-1-M7)172.jpg
./train/HTC-1-M7/(HTC-1-M7)100.jpg
./train/HTC-1-M7/(HTC-1-M7)121.jpg
./train/HTC-1-M7/(HTC-1-M7)20.jpg
./train/HTC-1-M7/(HTC-1-M7)132.jpg
./train/HTC-1-M7/(HTC-1-M7)184.jpg
./train/HTC-1-M7/(HTC-1-M7)88.jpg
./train/HTC-1-M7/(HTC-1-M7)209.jpg
./train/HTC-1-M7/(HTC-1-M7)59.jpg
./train/HTC-1-M7/(HTC-1-M7)87.jpg
./train/HTC-1-M7/(HTC-1-M7)74.jpg
./train/HTC-1-M7/(HTC-1-M7)122.jpg
./train/HTC-1-M7/(HTC-1-M7)64.jpg
./train/HTC-1-M7/(HTC-1-M7)207.jpg
./train/HTC-1-M7/(HTC-1-M7)106.jpg
./train/HTC-1-M7/(HTC-1-M7)164.jpg
./train/HTC-1-M7/(HTC-1-M7)226.jpg
./train/HTC-1-M7/(HTC-1-M7)39.jpg
./train/HTC-1-M7/(HTC-1-M7)270.jpg
./train/HTC-1-M7/(HTC-1-M7)155.jpg
./train/HTC-1-M7/(HTC-1-M7)16

In [105]:
list_paths = []
for subdir, dirs, files in os.walk("."):
    for file in files:
#         print(os.path.join(subdir, file))
        filepath = subdir + os.sep + file
        list_paths.append(filepath)

In [113]:
list_train = [filepath for filepath in list_paths if "train/" in filepath]
shuffle(list_train)

list_test = [filepath for filepath in list_paths if "test/" in filepath]
list_train = list_train
list_test = list_test
index = [os.path.basename(filepath) for filepath in list_test]

In [123]:
list_classes = list(set([os.path.dirname(filepath).split(os.sep)[-1] for filepath in list_paths if "train" in filepath]))

In [125]:
def get_class_from_path(filepath):
    return os.path.dirname(filepath).split(os.sep)[-1]

def read_and_resize(filepath):
    im_array = np.array(Image.open((filepath)), dtype="uint8")
    pil_im = Image.fromarray(im_array)
    new_array = np.array(pil_im.resize((256, 256)))
    return new_array/255

def label_transform(labels):
    labels = pd.get_dummies(pd.Series(labels))
    label_index = labels.columns.values

    return labels, label_index

In [126]:
X_train = np.array([read_and_resize(filepath) for filepath in list_train])

In [131]:
X_test = np.array([read_and_resize(filepath) for filepath in list_test])

In [134]:
labels = [get_class_from_path(filepath) for filepath in list_train]
y, label_index = label_transform(labels)
y = np.array(y)

In [136]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalMaxPool2D, Concatenate
input_shape = (256, 256, 3)
nclass = len(label_index)
def get_model():

    nclass = len(label_index)
    inp = Input(shape=input_shape)
    norm_inp = BatchNormalization()(inp)
    img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu, padding="same")(norm_inp)
    img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = MaxPooling2D(pool_size=(3, 3))(img_1)
    img_1 = Dropout(rate=0.2)(img_1)
    img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu, padding="same")(img_1)
    img_1 = MaxPooling2D(pool_size=(3, 3))(img_1)
    img_1 = Dropout(rate=0.2)(img_1)
    img_1 = Convolution2D(64, kernel_size=2, activation=activations.relu, padding="same")(img_1)
    img_1 = Convolution2D(20, kernel_size=2, activation=activations.relu, padding="same")(img_1)
    img_1 = GlobalMaxPool2D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)
    dense_1 = Dense(20, activation=activations.relu)(img_1)
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam()

    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

Using TensorFlow backend.


In [137]:
model = get_model()
file_path="weights.best.hdf5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor="val_acc", mode="max", patience=1)

callbacks_list = [checkpoint, early] #early

history = model.fit(X_train, y, validation_split=0.1, epochs=3, shuffle=True, verbose=2,
                              callbacks=callbacks_list)

#print(history)

model.load_weights(file_path)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 85, 85, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 85, 85, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 85, 85, 32)        4640      
__________

In [138]:
predicts = model.predict(X_test)
predicts = np.argmax(predicts, axis=1)
predicts = [label_index[p] for p in predicts]

In [140]:
df = pd.DataFrame(columns=['fname', 'camera'])

In [142]:
df['fname'] = index
df['camera'] = predicts

In [143]:
df

,fname,camera
0,img_2ea3452_manip.tif,Sony-NEX-7
1,img_06a8da2_unalt.tif,Samsung-Galaxy-S4
2,img_e6cf62a_manip.tif,Sony-NEX-7
3,img_3399f2a_manip.tif,Sony-NEX-7
4,img_a9c0694_unalt.tif,Sony-NEX-7
5,img_3c6976b_manip.tif,Sony-NEX-7
6,img_0d14e4e_unalt.tif,Sony-NEX-7
7,img_6345cc6_manip.tif,Samsung-Galaxy-S4
8,img_beb5043_unalt.tif,Sony-NEX-7
9,img_9c36cb2_manip.tif,Sony-NEX-7
